In [1]:
import pandas as pd
import numpy as np

In [2]:
from vnpy_portfoliostrategy.helperclass import *

In [61]:
# Create example pos_data (current positions)
pos_data = pd.Series([10,-5,0], index = ['fu2405.SHFE','fu2405.SHFE','fu2407.SHFE'])

# Create example to_trade_df (target positions)
to_trade_df = pd.DataFrame({
    'symbol': ['fu2405.SHFE', 'fu2406.SHFE', 'fu2408.SHFE'],  # Note: fu2408 is new position
    'target': ['15', '-8', '3']  # Strings because that's how they come from DB split()
})

# Now let's run through the operations to see the result:
print("Original pos_data:")
print(pos_data)
print("\nOriginal to_trade_df:")
print(to_trade_df)

# Apply the operations from your code
to_trade_df['target'] = pd.to_numeric(to_trade_df['target'])
ans = pos_data.rename('pos').to_frame().join(to_trade_df.drop_duplicates().set_index('symbol'),how='outer')
ans = ans.replace(0,np.nan).dropna(how='all').replace(np.nan,0)

print("\nFinal result (ans):")
print(ans)

Original pos_data:
fu2405.SHFE    10
fu2405.SHFE    -5
fu2407.SHFE     0
dtype: int64

Original to_trade_df:
        symbol target
0  fu2405.SHFE     15
1  fu2406.SHFE     -8
2  fu2408.SHFE      3

Final result (ans):
              pos  target
fu2405.SHFE  10.0    15.0
fu2405.SHFE  -5.0    15.0
fu2406.SHFE   0.0    -8.0
fu2408.SHFE   0.0     3.0


In [73]:
result = defaultdict(lambda: defaultdict(list))
# Add values while preserving duplicates
for index, row in ans.iterrows():
    result['pos'][index].append(row['pos'])
    result['target'][index].append(row['target']) if not row['target'] in result['target'][index] else None

In [79]:
import json
json.loads(json.dumps(result))


{'pos': {'fu2405.SHFE': [10.0, -5.0],
  'fu2406.SHFE': [0.0],
  'fu2408.SHFE': [0.0]},
 'target': {'fu2405.SHFE': [15.0],
  'fu2406.SHFE': [-8.0],
  'fu2408.SHFE': [3.0]}}

In [52]:
ans

{'pos': {'fu2405.SHFE': <vnpy_portfoliostrategy.helperclass.PositionInfo at 0x27007ba6f80>,
  'fu2406.SHFE': 0.0,
  'fu2408.SHFE': 0.0},
 'target': {'fu2405.SHFE': 15.0, 'fu2406.SHFE': -8.0, 'fu2408.SHFE': 3.0}}

In [44]:
ans['pos']['fu2405.SHFE']

-5.0

In [43]:
sorted(ans['pos']['fu2405.SHFE'].values.reshape(1,-1)[0].tolist())

AttributeError: 'float' object has no attribute 'values'

In [41]:
sorted(ans[['pos']].loc['fu2406.SHFE'].values.reshape(1,-1)[0].tolist())

[0.0]

In [34]:
PositionInfo(ans[['pos']].loc['fu2406.SHFE'].values.reshape(1,-1)[0].totuple()).net_pos()

AttributeError: 'numpy.ndarray' object has no attribute 'totuple'

In [5]:
PositionInfo.from_long_short(10,5).net_pos()

5

In [13]:
trading_schedule

,today,date,symbol,strategy,sc_symbol
0,2024-12-18,2024-12-19,"a2505.DCE,al2502.SHFE,ao2501.SHFE,AP505.CZCE,b...",strategy2,dom


In [14]:
trading_hours

,symbol,trading_hours,timezone
0,a.DCE,"21:01-23:00,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
1,ag.SHFE,"21:01-02:30,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
2,al.SHFE,"21:01-01:00,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
3,ao.SHFE,"21:01-01:00,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
4,AP.CZCE,"09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
...,...,...,...
76,WH.CZCE,"09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
77,wr.SHFE,"09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
78,y.DCE,"21:01-23:00,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai
79,ZC.CZCE,"21:01-23:00,09:01-10:15,10:31-11:30,13:31-15:00",Asia/Shanghai


In [12]:
rebal_tar

,symbol,target
0,A,0
1,AL,-2
2,AO,0
3,AP,0
4,B,0
5,BU,0
6,C,0
7,CF,0
8,CS,0
9,CY,0


In [14]:
to_trade_df['target'] = pd.to_numeric(to_trade_df['target'])

In [15]:
ans = pos_data[['symbol','pos']].set_index('symbol').join(to_trade_df.set_index('symbol_y'),how='outer')

In [17]:
ans.index

Index(['AP501.CZCE', 'AP501.CZCE', 'CF501.CZCE', 'CF501.CZCE', 'CY503.CZCE',
       'CY503.CZCE', 'FG501.CZCE', 'FG501.CZCE', 'MA501.CZCE', 'MA501.CZCE',
       'OI501.CZCE', 'OI501.CZCE', 'PF502.CZCE', 'PF502.CZCE', 'PK501.CZCE',
       'PK501.CZCE', 'PR503.CZCE', 'PR503.CZCE', 'PX501.CZCE', 'PX501.CZCE',
       'RM501.CZCE', 'RM501.CZCE', 'SA501.CZCE', 'SA501.CZCE', 'SF501.CZCE',
       'SF501.CZCE', 'SM501.CZCE', 'SM501.CZCE', 'SR501.CZCE', 'SR501.CZCE',
       'TA501.CZCE', 'TA501.CZCE', 'UR501.CZCE', 'UR501.CZCE', 'a2501.DCE',
       'a2501.DCE', 'al2501.SHFE', 'al2501.SHFE', 'ao2501.SHFE', 'ao2501.SHFE',
       'b2501.DCE', 'b2501.DCE', 'bu2501.SHFE', 'bu2501.SHFE', 'c2501.DCE',
       'c2501.DCE', 'cs2501.DCE', 'cs2501.DCE', 'eb2412.DCE', 'eb2412.DCE',
       'eg2501.DCE', 'eg2501.DCE', 'fu2501.SHFE', 'fu2501.SHFE', 'hc2501.SHFE',
       'hc2501.SHFE', 'i2501.DCE', 'i2501.DCE', 'j2501.DCE', 'j2501.DCE',
       'jd2501.DCE', 'jd2501.DCE', 'jm2501.DCE', 'jm2501.DCE', 'l2501.DCE',


In [18]:
set(pd.Series(list(set(pos_data['symbol'].values.tolist()) | set(to_trade_df['symbol_y'].values.tolist()))).str.strip().values.tolist()) == set(ans.index)

True

In [21]:
import json

In [33]:
str(datetime.now())

'2024-09-23 14:31:16.126480'

In [11]:
pd.concat([pd.Series(trading_schedule['symbol'].values[0].split(',')).str[:-4],
                            pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,keys=['symbol','symb']
                        ).merge(trading_hours[['rqsymbol','symbol']],left_on='symb',right_on='rqsymbol',how='inner'
                                )

,symbol_x,symb,rqsymbol,symbol_y
0,A,A2501,A2501,a2501.DCE
1,AL,AL2411,AL2411,al2411.SHFE
2,AO,AO2411,AO2411,ao2411.SHFE
3,AP,AP2501,AP2501,AP501.CZCE
4,B,B2411,B2411,b2411.DCE
...,...,...,...,...
62,WH,WH2411,WH2411,WH411.CZCE
63,WR,WR2410,WR2410,wr2410.SHFE
64,Y,Y2501,Y2501,y2501.DCE
65,ZC,ZC2410,ZC2410,ZC410.CZCE


In [12]:
rollover_df.query('pre!=post')

,pre,post


In [10]:
rollover_df = pd.concat([pd.Series(previous_trading_schedule['symbol'].values[0].split(',')),
                 pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,keys=['pre','post'])

In [20]:
rollover_df.query("pre!=post").replace(trading_hours[['rqsymbol','symbol']].set_index('rqsymbol').to_dict()['symbol'])

,pre,post
63,WR2409,wr2410.SHFE


In [19]:
rollover_df.query("pre!=post").map(lambda x: len(str(x)))

,pre,post
63,WR2409,WR2410


In [15]:
trading_hours[['rqsymbol','symbol']].set_index('rqsymbol').to_dict()['symbol']

{'A2501': 'a2501.DCE',
 'AG2412': 'ag2412.SHFE',
 'AL2411': 'al2411.SHFE',
 'AO2411': 'ao2411.SHFE',
 'AP2501': 'AP501.CZCE',
 'AU2412': 'au2412.SHFE',
 'B2411': 'b2411.DCE',
 'BB2501': 'bb2501.DCE',
 'BC2411': 'bc2411.INE',
 'BR2411': 'br2411.SHFE',
 'BU2411': 'bu2411.SHFE',
 'C2411': 'c2411.DCE',
 'CF2501': 'CF501.CZCE',
 'CJ2501': 'CJ501.CZCE',
 'CS2411': 'cs2411.DCE',
 'CU2410': 'cu2410.SHFE',
 'CY2501': 'CY501.CZCE',
 'EB2411': 'eb2411.DCE',
 'EC2412': 'ec2412.INE',
 'EG2501': 'eg2501.DCE',
 'FB2501': 'fb2501.DCE',
 'FG2501': 'FG501.CZCE',
 'FU2411': 'fu2411.SHFE',
 'HC2501': 'hc2501.SHFE',
 'I2501': 'i2501.DCE',
 'IC2410': 'IC2410.CFFEX',
 'IF2410': 'IF2410.CFFEX',
 'IH2410': 'IH2410.CFFEX',
 'IM2410': 'IM2410.CFFEX',
 'J2501': 'j2501.DCE',
 'JD2501': 'jd2501.DCE',
 'JM2501': 'jm2501.DCE',
 'JR2411': 'JR411.CZCE',
 'L2501': 'l2501.DCE',
 'LC2411': 'lc2411.GFEX',
 'LH2411': 'lh2411.DCE',
 'LR2411': 'LR411.CZCE',
 'LU2412': 'lu2412.INE',
 'M2501': 'm2501.DCE',
 'MA2501': 'MA501.CZC